In [1]:
import os
import yaml
import time
import torch
import epynet
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from utils.epanet_loader import get_nx_graph
from utils.epanet_simulator import epanetSimulator
from utils.data_loader import battledimLoader, dataCleaner, dataGenerator, embedSignalOnGraph, rescaleSignal
from modules.torch_gnn import ChebNet
from utils.visualisation import visualise

In [2]:
def predict_pressure(graph, pressure_series, print_out_rate=100, save=True, filename='reconstructions.csv'):
    
    results = []
    elapsed_time = time.time()
    
    for i, partial_graph_signal in enumerate(pressure_series):
        if not i % print_out_rate:
            execution_time = time.time()
        
        results.append(model.predict(graph, partial_graph_signal))
        
        if not i % print_out_rate:
            print('Signal:\t{}\t Execution:\t{:.3f} s\t Elapsed:\t{:.3f} s'.format(i,
                                                                                   time.time()-execution_time, 
                                                                                   time.time()-elapsed_time))
    if save:
        print('-'*63+"\nSaving results to: \n{}/{}\n\n".format(os.getcwd(),filename))
        pd.DataFrame(results).to_csv(filename)
    
    return results

In [3]:
# Runtime configuration
path_to_wdn     = './data/L-TOWN.inp'
path_to_data    = './data/l-town-data/'
weight_mode     = 'pipe_length'
self_loops      = True
scaling         = 'minmax'
figsize         = (50,16)
print_out_rate  = 100               
model_name      = 'l-town-chebnet-' + weight_mode +'-' + scaling + '{}'.format('-self_loop' if self_loops else '')
last_model_path = './studies/models/l-town-chebnet-pipe_length-minmax-self_loop-1.pt'
last_log_path   = './studies/logs/l-town-chebnet-pipe_length-minmax-self_loop-1.csv' 

In [4]:
print('1. Making graph from EPANET input file \n')
    
# Import the .inp file using the EPYNET library
wdn = epynet.Network(path_to_wdn)

# Solve hydraulic model for a single timestep
wdn.solve()

# Convert the file using a custom function, based on:
# https://github.com/BME-SmartLab/GraphConvWat 
G , pos , head = get_nx_graph(wdn, weight_mode=weight_mode, get_head=True)

# Open the dataset configuration file
with open(path_to_data + 'dataset_configuration.yaml') as file:

    # Load the configuration to a dictionary
    config = yaml.load(file, Loader=yaml.FullLoader) 

# Generate a list of integers, indicating the number of the node
# at which a  pressure sensor is present
sensors = [int(string.replace("n", "")) for string in config['pressure_sensors']]

if self_loops:
    for sensor_node in sensors:             # For each node in the sensor list
        G.add_edge(u_of_edge=sensor_node,   # Add an edge from that node ...
                    v_of_edge=sensor_node,   # ... to itself ...
                    weight=1.,name='SELF')   # ... and set its weight to equal 1

1. Making graph from EPANET input file 



In [5]:
# Instantiate the nominal WDN model
nominal_wdn_model = epanetSimulator(path_to_wdn, path_to_data)

# Run a simulation
nominal_wdn_model.simulate()

# Retrieve the nodal pressures
nominal_pressure = nominal_wdn_model.get_simulated_pressure()

In [6]:
x,y,scale,bias = dataCleaner(pressure_df    = nominal_pressure, # Pass the nodal pressures
                             observed_nodes = sensors,          # Indicate which nodes have sensors
                             rescale        = scaling)          # Perform scaling on the timeseries data

# Split the data into training and validation sets
x_trn, x_val, y_trn, y_val = train_test_split(x, y, 
                                              test_size    = 0.2,
                                              random_state = 1,
                                              shuffle      = False)

In [7]:
print('Importing SCADA dataset...\n') 
        
# Load the data into a numpy array with format matching the GraphConvWat problem
pressure_2018 = battledimLoader(observed_nodes = sensors,
                                n_nodes        = 782,
                                path           = path_to_data,
                                file           = '2018_SCADA_Pressures.csv',
                                rescale        = True, 
                                scale          = scale,
                                bias           = bias)

pressure_2019 = battledimLoader(observed_nodes = sensors,
                                n_nodes        = 782,
                                path           = path_to_data,
                                file           = '2019_SCADA_Pressures.csv',
                                rescale        = True, 
                                scale          = scale,
                                bias           = bias)

Importing SCADA dataset...



In [8]:
pressure_2018.shape

(105120, 782, 2)

In [9]:
# Set the computation device as NVIDIA GPU if available else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Instantiate a Chebysev Network GNN model
model  = ChebNet(name           = 'ChebNet',
                 data_generator = None,
                 device         = device, 
                 in_channels    = np.shape(x_trn)[-1], 
                 out_channels   = np.shape(y_trn)[-1],
                 data_scale     = scale, 
                 data_bias      = bias).to(device)

In [10]:
model.load_model(last_model_path, last_log_path)


                Loaded previous model results...
                --------------------------------------------------
                Model has been trained for:	100 epochs
                Best validation loss:      	2.8486794995296693e-05 
                Occurred in training round:	74 


C:\Users\30806\OneDrive\UCL\project\Project\Training model\modules\torch_gnn.py:302: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.load_state_dict(torch.load(path_to_mo

In [ ]:
test = pressure_2018[:1000,:,:]

In [ ]:
test.shape

In [11]:
print('\n')
print('5.1 Predicting 2018 pressure\n')

prediction_2018 = predict_pressure(G, 
                                    pressure_2018,
                                    print_out_rate = print_out_rate, 
                                    save           = True, 
                                    filename       = './studies/results/recon_pressure_2018.csv')



5.1 Predicting 2018 pressure

Signal:	0	 Execution:	3.034 s	 Elapsed:	3.034 s
Signal:	100	 Execution:	0.136 s	 Elapsed:	17.638 s
Signal:	200	 Execution:	0.137 s	 Elapsed:	31.627 s
Signal:	300	 Execution:	0.137 s	 Elapsed:	45.262 s
Signal:	400	 Execution:	0.135 s	 Elapsed:	59.107 s
Signal:	500	 Execution:	0.140 s	 Elapsed:	73.049 s
Signal:	600	 Execution:	0.138 s	 Elapsed:	86.918 s
Signal:	700	 Execution:	0.138 s	 Elapsed:	100.741 s
Signal:	800	 Execution:	0.134 s	 Elapsed:	114.390 s
Signal:	900	 Execution:	0.134 s	 Elapsed:	128.371 s
Signal:	1000	 Execution:	0.136 s	 Elapsed:	142.252 s
Signal:	1100	 Execution:	0.136 s	 Elapsed:	156.185 s
Signal:	1200	 Execution:	0.133 s	 Elapsed:	169.968 s
Signal:	1300	 Execution:	0.132 s	 Elapsed:	183.581 s
Signal:	1400	 Execution:	0.136 s	 Elapsed:	197.391 s
Signal:	1500	 Execution:	0.139 s	 Elapsed:	211.282 s
Signal:	1600	 Execution:	0.133 s	 Elapsed:	225.062 s
Signal:	1700	 Execution:	0.134 s	 Elapsed:	238.880 s
Signal:	1800	 Execution:	0.138 s	 

In [12]:
print('\n')
print('5.1 Predicting 2018 pressure\n')

prediction_2019 = predict_pressure(G, 
                                    pressure_2019,
                                    print_out_rate = print_out_rate, 
                                    save           = True, 
                                    filename       = './studies/results/recon_pressure_2019.csv')



5.1 Predicting 2018 pressure

Signal:	0	 Execution:	0.179 s	 Elapsed:	0.179 s
Signal:	100	 Execution:	0.137 s	 Elapsed:	14.309 s
Signal:	200	 Execution:	0.136 s	 Elapsed:	28.346 s
Signal:	300	 Execution:	0.136 s	 Elapsed:	42.496 s
Signal:	400	 Execution:	0.135 s	 Elapsed:	56.354 s
Signal:	500	 Execution:	0.134 s	 Elapsed:	70.635 s
Signal:	600	 Execution:	0.137 s	 Elapsed:	84.714 s
Signal:	700	 Execution:	0.137 s	 Elapsed:	98.580 s
Signal:	800	 Execution:	0.136 s	 Elapsed:	112.808 s
Signal:	900	 Execution:	0.424 s	 Elapsed:	127.011 s
Signal:	1000	 Execution:	0.139 s	 Elapsed:	140.861 s
Signal:	1100	 Execution:	0.135 s	 Elapsed:	154.930 s
Signal:	1200	 Execution:	0.135 s	 Elapsed:	168.983 s
Signal:	1300	 Execution:	0.137 s	 Elapsed:	183.121 s
Signal:	1400	 Execution:	0.135 s	 Elapsed:	197.104 s
Signal:	1500	 Execution:	0.134 s	 Elapsed:	210.857 s
Signal:	1600	 Execution:	0.146 s	 Elapsed:	225.105 s
Signal:	1700	 Execution:	0.137 s	 Elapsed:	239.196 s
Signal:	1800	 Execution:	0.135 s	 E